In [1]:
import itertools
import pandas as pd
import numpy as np
import csv

In [2]:
file_path = './data/test_tabulation_area.xtb.tbw'
dic_path = './data/CensusMainShortForm.dcf'

f = open(file_path, 'r')


In [3]:
dimension = []
all_values = []
all_codes= []
all_rows = []
values = []
area_names = []
area_codes = []

In [4]:
i=1
notFirst=False
while True:
        
        line = f.readline()
        if not line:
            if notFirst:
                all_values.append(values)
            break
        else:
            if line.find('[Variable]') != -1:
                if notFirst:
                    all_values.append(values)
                name = f.readline()
                name = name.replace("\n", "")
                name =  name[5:]
                dimension.append(name)
                values = []
                notFirst = True
            if line.find('[Value]') != -1:
                label = f.readline()
                label = label.replace("\n", "")
                label = label[6:]
                #print(label)
                values.append(label)
            if line.find('AreaName') == 0: 
                area = line[9:]
                #print(area)
                area_names.append(area.strip())
            if line.find('BreakKey') == 0: 
                code = line[9:]
                code = code.replace(';', '')
                #print(area)
                area_codes.append(code.strip())
            if line.find('Row') == 0: 
                #row = f.readline()
                row = line[4:]
                i = i+1
                arr = [ x.strip() for x in row.split('  ')]
                all_rows += arr
              

               

In [5]:
len(all_rows)

12576

In [6]:
#print(area_codes)              
#all_values[0] = area_codes                
#print(dimension)
#print(area_names)
#print(all_values)
#print(all_rows)

In [7]:
print(all_values[0])

['%AreaName%']


In [8]:
d = open(dic_path, 'r', encoding='utf-8')
header =['cod','desc', 'cod_list']
dim_values = []
total = []
a =  np.empty((0, 3))

def getCodes(d, dim):
    d.seek(0)
    values = []
    dim=dim + "\n"
    while True:
        line = d.readline()
        if not line:
            break
        if line.find(dim) != -1:
            line = d.readline()
            while line.find('Value') != -1:
                line = line.replace("\n", "")
                line = line[6:]
                line = line + ';' + dim
                item = line.split(';')
                values.append(item)
                line = d.readline()
                if line.find('Note=') != -1:
                    line = d.readline()
    return values


for dim in dimension[1:]:
    values = getCodes(d, dim)
    total = ['0', 'Total', dim]
    values.append(total)
    print(values)
    print("\n")
    a = np.concatenate((a, values), axis=0)

[['1', 'Male', 'P11_VS1\n'], ['2', 'Female', 'P11_VS1\n'], ['3', 'Intersex', 'P11_VS1\n'], ['0', 'Total', 'P11_VS1']]


[['1', 'Never Married', 'P18_VS1\n'], ['2', 'Married Monogamous', 'P18_VS1\n'], ['3', 'Married Polygamous', 'P18_VS1\n'], ['4', 'Widowed', 'P18_VS1\n'], ['5', 'Divorced', 'P18_VS1\n'], ['6', 'Separated', 'P18_VS1\n'], ['9', 'DK', 'P18_VS1\n'], ['0', 'Total', 'P18_VS1']]




In [9]:
a

array([['1', 'Male', 'P11_VS1\n'],
       ['2', 'Female', 'P11_VS1\n'],
       ['3', 'Intersex', 'P11_VS1\n'],
       ['0', 'Total', 'P11_VS1'],
       ['1', 'Never Married', 'P18_VS1\n'],
       ['2', 'Married Monogamous', 'P18_VS1\n'],
       ['3', 'Married Polygamous', 'P18_VS1\n'],
       ['4', 'Widowed', 'P18_VS1\n'],
       ['5', 'Divorced', 'P18_VS1\n'],
       ['6', 'Separated', 'P18_VS1\n'],
       ['9', 'DK', 'P18_VS1\n'],
       ['0', 'Total', 'P18_VS1']], dtype='<U32')

In [10]:
area_varname = [dimension[0]] * len(area_codes)


In [11]:
b = np.column_stack((area_codes, area_names, area_varname))
print(len(b))

393


In [12]:
c = np.concatenate((b, a), axis=0)

codes = pd.DataFrame(c)

codes.columns =  header

codes

,cod,desc,cod_list
0,,Kenya,SYSTEM_TOTAL
1,01,Mombasa,SYSTEM_TOTAL
2,0101,Changamwe,SYSTEM_TOTAL
3,0102,Jomvu,SYSTEM_TOTAL
4,0103,Kisauni,SYSTEM_TOTAL
...,...,...,...
400,4,Widowed,P18_VS1\n
401,5,Divorced,P18_VS1\n
402,6,Separated,P18_VS1\n
403,9,DK,P18_VS1\n


In [13]:
codes['cod_list'].replace(regex=True,inplace=True,to_replace=r'\n',value=r'')
codes

,cod,desc,cod_list
0,,Kenya,SYSTEM_TOTAL
1,01,Mombasa,SYSTEM_TOTAL
2,0101,Changamwe,SYSTEM_TOTAL
3,0102,Jomvu,SYSTEM_TOTAL
4,0103,Kisauni,SYSTEM_TOTAL
...,...,...,...
400,4,Widowed,P18_VS1
401,5,Divorced,P18_VS1
402,6,Separated,P18_VS1
403,9,DK,P18_VS1


In [14]:
all_codes = []
all_codes.append(area_codes)

for i, dim in enumerate(dimension[1:]):
    #print(i, dim)
    arr_cod = []
    for arr in all_values[i+1]:

        d =  ''
        result = codes.loc[(codes['cod_list'] == dim) & (codes['desc'] == arr)]
        try:
            d = result['cod'].values[0]
            arr_cod.append(d)
        except:
            pass
    print(len(arr_cod))
    all_codes.append(arr_cod)
        




4
8


In [15]:
all_codes

[['',
  '01',
  '0101',
  '0102',
  '0103',
  '0104',
  '0105',
  '0106',
  '02',
  '0201',
  '0202',
  '0203',
  '0204',
  '0205',
  '03',
  '0301',
  '0302',
  '0303',
  '0304',
  '0305',
  '0306',
  '0307',
  '0308',
  '0309',
  '04',
  '0401',
  '0402',
  '0403',
  '05',
  '0501',
  '0502',
  '06',
  '0601',
  '0602',
  '0603',
  '0604',
  '07',
  '0701',
  '0702',
  '0703',
  '0704',
  '0705',
  '0706',
  '0707',
  '08',
  '0801',
  '0802',
  '0803',
  '0804',
  '0805',
  '0806',
  '0807',
  '0808',
  '09',
  '0901',
  '0902',
  '0903',
  '0904',
  '0905',
  '0906',
  '0907',
  '10',
  '1001',
  '1002',
  '1003',
  '1004',
  '1005',
  '1006',
  '1007',
  '11',
  '1101',
  '1102',
  '1103',
  '12',
  '1201',
  '1202',
  '1203',
  '1204',
  '1205',
  '1206',
  '1207',
  '1208',
  '1209',
  '1210',
  '1211',
  '1212',
  '1213',
  '13',
  '1301',
  '1302',
  '1303',
  '1304',
  '1305',
  '1306',
  '14',
  '1401',
  '1402',
  '1403',
  '1404',
  '1405',
  '1406',
  '15',
  '1501',
  '1

In [16]:
result = list(itertools.product(*all_codes))

print(len(result))

12576


In [17]:
print(result)

[('', '0', '0'), ('', '0', '1'), ('', '0', '2'), ('', '0', '3'), ('', '0', '4'), ('', '0', '5'), ('', '0', '6'), ('', '0', '9'), ('', '1', '0'), ('', '1', '1'), ('', '1', '2'), ('', '1', '3'), ('', '1', '4'), ('', '1', '5'), ('', '1', '6'), ('', '1', '9'), ('', '2', '0'), ('', '2', '1'), ('', '2', '2'), ('', '2', '3'), ('', '2', '4'), ('', '2', '5'), ('', '2', '6'), ('', '2', '9'), ('', '3', '0'), ('', '3', '1'), ('', '3', '2'), ('', '3', '3'), ('', '3', '4'), ('', '3', '5'), ('', '3', '6'), ('', '3', '9'), ('01', '0', '0'), ('01', '0', '1'), ('01', '0', '2'), ('01', '0', '3'), ('01', '0', '4'), ('01', '0', '5'), ('01', '0', '6'), ('01', '0', '9'), ('01', '1', '0'), ('01', '1', '1'), ('01', '1', '2'), ('01', '1', '3'), ('01', '1', '4'), ('01', '1', '5'), ('01', '1', '6'), ('01', '1', '9'), ('01', '2', '0'), ('01', '2', '1'), ('01', '2', '2'), ('01', '2', '3'), ('01', '2', '4'), ('01', '2', '5'), ('01', '2', '6'), ('01', '2', '9'), ('01', '3', '0'), ('01', '3', '1'), ('01', '3', '2'), (

In [18]:
df = pd.DataFrame(result)
df.columns = dimension
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12576 entries, 0 to 12575
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   SYSTEM_TOTAL  12576 non-null  object
 1   P11_VS1       12576 non-null  object
 2   P18_VS1       12576 non-null  object
dtypes: object(3)
memory usage: 294.9+ KB


In [19]:
len(all_rows)


12576

In [20]:
df['VALUES'] = all_rows
df['VALUES'] = df['VALUES'].str.replace(',000000000', '')
df

,SYSTEM_TOTAL,P11_VS1,P18_VS1,VALUES
0,,0,0,3251302
1,,0,1,1416373
2,,0,2,1523590
3,,0,3,95030
4,,0,4,114747
...,...,...,...,...
12571,4711,3,3,1
12572,4711,3,4,
12573,4711,3,5,
12574,4711,3,6,


In [21]:
codes

,cod,desc,cod_list
0,,Kenya,SYSTEM_TOTAL
1,01,Mombasa,SYSTEM_TOTAL
2,0101,Changamwe,SYSTEM_TOTAL
3,0102,Jomvu,SYSTEM_TOTAL
4,0103,Kisauni,SYSTEM_TOTAL
...,...,...,...
400,4,Widowed,P18_VS1
401,5,Divorced,P18_VS1
402,6,Separated,P18_VS1
403,9,DK,P18_VS1


In [22]:
d = codes.loc[codes['cod_list']=='SYSTEM_TOTAL']
d

,cod,desc,cod_list
0,,Kenya,SYSTEM_TOTAL
1,01,Mombasa,SYSTEM_TOTAL
2,0101,Changamwe,SYSTEM_TOTAL
3,0102,Jomvu,SYSTEM_TOTAL
4,0103,Kisauni,SYSTEM_TOTAL
...,...,...,...
388,4707,Makadara,SYSTEM_TOTAL
389,4708,Mathare,SYSTEM_TOTAL
390,4709,Njiru,SYSTEM_TOTAL
391,4710,Starehe,SYSTEM_TOTAL


In [23]:
for cod in dimension:
    
    cod_col = 'DESC_'+cod
    print(cod)
    d = codes.loc[codes['cod_list']==cod]
    df = pd.merge(df, d, how='inner', left_on=cod, right_on='cod')
    df = df.drop('cod', axis=1)
    df = df.drop('cod_list', axis=1)
    df.rename(columns={'desc': cod_col}, inplace=True) 
df

SYSTEM_TOTAL
P11_VS1
P18_VS1


,SYSTEM_TOTAL,P11_VS1,P18_VS1,VALUES,DESC_SYSTEM_TOTAL,DESC_P11_VS1,DESC_P18_VS1
0,,0,0,3251302,Kenya,Total,Total
1,01,0,0,86773,Mombasa,Total,Total
2,0101,0,0,9768,Changamwe,Total,Total
3,0102,0,0,11766,Jomvu,Total,Total
4,0103,0,0,20275,Kisauni,Total,Total
...,...,...,...,...,...,...,...
12571,4707,3,9,,Makadara,Intersex,DK
12572,4708,3,9,,Mathare,Intersex,DK
12573,4709,3,9,,Njiru,Intersex,DK
12574,4710,3,9,,Starehe,Intersex,DK


In [24]:
df.to_csv('./out.csv', index=False)